In [1]:
number_of_shots = 5

In [2]:
experiment_name = "2024.05.07-02_Mixtral_7B_instruct_API_5-few-shots_RANDOM"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mixtral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx").drop(columns=['Unnamed: 0'])
data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
    (data_sel_4_few_shot_examples.text_id==5614) |
    (data_sel_4_few_shot_examples.text_id==2089) |
    (data_sel_4_few_shot_examples.text_id==5665) |
    (data_sel_4_few_shot_examples.text_id==370)  |
    (data_sel_4_few_shot_examples.text_id==1422) # |
#     (data_sel_4_few_shot_examples.text_id==4329) |
#     (data_sel_4_few_shot_examples.text_id==4929) 

]

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
738,738,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000,96
1092,1092,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778,76
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000,136


In [10]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [12]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are {number_of_shots} examples of expected classification given an input text:

"""+few_shots+"""

Answer with the label only using the format:---LABEL---
"""


In [13]:
data_sel = data #.iloc[:5]

In [14]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11--- (This text pertains to infrastructure and safety in urban areas, which aligns with SDG 11: Sustainable Cities and Communities.)
1 6099 0  ---SDG 11--- (Sustainable Cities and Communities) as the text describes a photo shoot in the Maldives, which can be related to promoting and raising awareness about sustainable tourism in vulnerable coastal communities.
2 6471 0  ---SDG 16---
3 1786 0  ---SDG 4--- (Goal 4: Quality Education) This text highlights the importance of education in the field of astronomy, with a specific focus on an Indian astronomer who has made significant contributions in this field.
4 2143 0  ---SDG 7---
5 3377 0  ---SDG 5---
6 5589 0  ---SDG 11--- (This text pertains to the entertainment and media industry, which can be classified under SDG 11's target 7: "by 2030, enhance inclusive and sustainable urbanization and capacity for participatory, integrated and sustainable human settlement planning and management").
7 5431 0  ---SDG 8---
(This text 

52 6243 0  ---SDG 16---
(Promotion of peaceful and inclusive societies for sustainable development, provision of access to justice for all and building effective, accountable and inclusive institutions at all levels) is the most appropriate SDG for the provided text, as it involves discussions on the Uniform Civil Code, politics, and the rights of a marginalized community.
53 4104 0  ---SDG 3--- (Goal 3 is about ensuring healthy lives and promoting well-being for all at all ages)
54 1497 0  ---SDG 8---
55 2055 0  ---SDG 3---  (This text is about health and discovery of new cell types, which aligns with the focus of SDG 3: Good Health and Well-being.)
56 2463 0  ---SDG 8---

The text discusses employment and career advancement in the context of sports management, which can be linked to the promotion of decent work and economic growth (SDG 8). However, it does not clearly align with any other specific SDG.
57 193 0  ---SDG 9--- (This text is about the progress of the Ariane 6 space launc

104 1733 1  ---SDG 8---
105 2694 1  ---SDG 1---

The text discusses the decomposition of changes in child poverty, which is a key aspect of the first Sustainable Development Goal (SDG) of ending poverty in all its forms everywhere. The decomposition analysis helps to measure the portion of changes in child poverty due to changes in the characteristics of child population and those due to changes in the association between these characteristics and poverty rates. This analysis is crucial for understanding the effectiveness of social transfers and policies in reducing child poverty and promoting inclusive economic growth.
106 6684 1  ---SDG 1--- (This text discusses poverty and unemployment, which are closely related to SDG 1: No Poverty.)
107 173 1  ---SDG 8---
108 6136 1  ---SDG 1--- (This text is about poverty and deprivation, which aligns with SDG 1: No Poverty.)
109 2652 1  ---SDG 1--- (SDG 1 refers to "No Poverty" and the text seems to be discussing issues related to poverty, speci

156 5694 2  ---SDG 2---
(This text discusses food security and small-scale agriculture, which aligns with SDG 2: Zero Hunger.)
157 4433 2  ---SDG 2--- (Goal 2 is "Zero Hunger" and the text discusses price behavior and production patterns of major producers in the cotton market)
158 654 2  ---SDG 2--- (This text is about food prices, agricultural output and their impact on poverty and malnutrition, which aligns with SDG 2's focus on zero hunger.)
159 4178 2  ---SDG 2---
160 5812 2  ---SDG 2--- (Goal 2 aims to end hunger, achieve food security and improved nutrition and promote sustainable agriculture.)
161 5130 2  ---SDG 2---
162 4078 2  ---SDG 8---
163 646 2  Based on the text provided, the experience of child food insecurity is being discussed, but it is not explicitly stated that the child is aware of it as such. The worry or hunger in this instance is a consequence of household food insecurity.

Therefore, the appropriate SDG label for this text would be:---SDG 2--- as it relates to

213 4897 3  ---SDG 3---
214 3854 3  ---SDG 3---
215 5666 3  ---SDG 3---

The text provided focuses on the evaluation of local health authorities using specific indicators in the domains of access, continuity, effectiveness, efficiency, patient satisfaction, and organizational competence. Many of these indicators reflect processes, such as vaccination or cancer screening coverage, while some effectiveness indicators reflect outcomes like adequate control of lipids and blood pressure in people with diabetes. The establishment of national targets for each indicator also highlights the focus on improving healthcare quality and performance. This content aligns with SDG 3, which aims to ensure healthy lives and promote well-being for all at all ages.
216 5064 3  ---SDG 3---
217 2952 3  ---SDG 3---
(This text is about the health care system of Finland, which relates to the overall health and well-being of the population, which is a target under SDG 3.)
218 499 3  ---SDG 3---
219 4673 3  ---SD

...

844 925 14  ---SDG 14---
845 4721 14  ---SDG 14--- (Sustainable Development Goal 14 is about life below water, and the text is about reducing pressures on global fisheries and expanding seafood exports, which aligns with this goal.)
846 6196 14  ---SDG 14--- (Life Below Water) as the text discusses issues related to the health of marine ecosystems and fisheries.
847 432 14  ---SDG 14--- (Sustainable use of marine resources)
848 4876 14  ---SDG 14--- (Life Below Water) as the text discusses the management and sustainability of fisheries, which connects to the SDG target 14.4 of effectively regulating harvesting and ending overfishing, illegal, unreported and unregulated fishing and destructive fishing practices.
849 785 14  ---SDG 14--- (Life Below Water)
850 3619 14  ---SDG 14---
851 4158 14  ---SDG 14--- (Life Below Water)
852 5523 14  ---SDG 14--- (Life Below Water) is the most appropriate SDG for this text as it focuses on the negative impact of international tensions on the marine 

902 4075 15  ---SDG 15---
903 3920 15  ---SDG 15--- (Life on Land) as the text discusses the protection of biological diversity and maintaining essential ecological processes, which aligns with the objectives of SDG 15.
904 794 15  ---SDG 15--- (Life on Land) as the text refers to the Amazon biome and its conservation.
905 78 15  ---SDG 15--- (Life on Land)
906 2788 15  ---SDG 15--- (Life on Land) as the text discusses enhancing capacity to map wildlife trafficking routes and mentions the CITES treaty which is about protecting endangered species.
907 3261 15  ---SDG 13--- (Climate Action) as the text discusses efforts and actions towards climate change mitigation, specifically the REDD+ agenda.
908 3358 15  ---SDG 15--- (Life on Land)
909 759 15  ---SDG 15--- (Life on Land) as the text discusses forestland management and afforestation.
910 1455 15  ---SDG 15--- (Goal 15 is about Life on Land, which includes conservation, restoration and sustainable use of terrestrial ecosystems. The te

949 3474 15  ---SDG 15--- (Life on Land) as the text is about biodiversity mainstreaming which is closely related to SDG 15's target of protecting, restoring and promoting the sustainable use of terrestrial ecosystems.
950 518 15  ---SDG 15--- (Life on Land)
951 2066 15  ---SDG 15--- (Life on Land) is the most appropriate SDG for this text, as it discusses the promotion of ecotourism and the exploration of instruments to capture the international public good benefits provided by protected areas, which aligns with SDG 15's focus on protecting, restoring, and promoting the sustainable use of terrestrial ecosystems.
952 389 15  ---SDG 15---
953 943 15  ---SDG 15--- (Life on Land) is the most appropriate SDG for this text, as it primarily discusses the loss of biodiversity, endemic and subendemic plant species, and invasive species, which are all directly related to the sustainable management and conservation of life on land.
954 6522 15  ---SDG 15---
955 3665 15  ---SDG 15--- (Life on Lan

977 6715 16  ---SDG 17--- (This text discusses international cooperation and law, which aligns with SDG 17: Partnerships for the Goals.)
978 2543 16  ---SDG 16---

Explanation:
The text discusses the role of quasi-judicial bodies in the enforcement of international humanitarian law, specifically in relation to Israel's naval blockade against the Mavi Marmara in 2010. This topic falls under SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all, and build effective, accountable, and inclusive institutions at all levels. The text highlights the benefits and potential problems of this new type of adjudication for international law and dispute resolution, which aligns with the objective of SDG 16 to promote the rule of law and access to justice.
979 64 16  ---SDG 16---
980 3614 16  ---SDG 16---
981 4520 16  ---SDG 3--- (This text discusses policies and services related to child mental health, which aligns with SDG 3's f

1010 535 16  ---SDG 8---
1011 2370 16  ---SDG 11---
1012 2870 16  ---SDG 16---

The text discusses various aspects of political philosophy, morality, and governance, which aligns closely with SDG 16: Peace, Justice and Strong Institutions. This goal aims to promote peaceful and inclusive societies, provide access to justice for all, and build effective, accountable, and inclusive institutions at all levels. The text discusses issues related to political morality, civic schooling, compassionate conservatism, and the role of institutions, which are all directly relevant to SDG 16.
1013 6330 16  ---SDG 16---

The text discusses the principle of subsidiarity in the judicial field, specifically in the context of human rights complaints in sub-regional courts in Africa. SDG 16 aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels. The principle of subsidiarity i

In [15]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [16]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9--- (This text is about supporting STI, which can be classified under SDG 9: Industry, Innovation, and Infrastructure.)')
('2', 6471, 0, ' ---SDG 16---')
('782', 2180, 13, ' ---SDG 11--- (targets related to sustainable cities and communities)')
('601', 4947, 10, ' ---SDG 10--- (Goal 10 is about reducing inequalities within and among countries, which seems to be the main topic of the text.)')
('144', 2727, 2, ' ---SDG 2--- (Sustainable Development Goal 2: Zero Hunger)')
('54', 1497, 0, ' ---SDG 8---')
('125', 5756, 2, ' ---SDG 11--- (This response is about sustainable cities and communities, as the text discusses rural and urban living standards, and the need for infrastructure investment and government assistance in the agricultural sector.)')
('611', 3928, 10, ' ---SDG 10--- (This text discusses economic inequality and redistribution, which aligns with SDG 10: Reduced Inequalities.)')
('354', 5927, 5, ' ---SDG 3--- (Goal 3 

In [17]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [18]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9--- (This text is about supporting ST...
2,2,6471,0,---SDG 16---
3,782,2180,13,---SDG 11--- (targets related to sustainable ...
4,601,4947,10,---SDG 10--- (Goal 10 is about reducing inequ...
...,...,...,...,...
1015,152,5700,2,---SDG 2--- (The text mentions food security ...
1016,276,1547,4,---SDG 4--- (Quality Education)
1017,554,5840,9,---SDG 8---
1018,927,6479,15,---SDG 15---


In [19]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (This text pertains to infrastru...
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- (Sustainable Cities and Communit...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 4--- (Goal 4: Quality Education) This ...
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 17---\n\n(This answer is based on the ...
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---


In [20]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (This text pertains to infrastru...,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- (Sustainable Cities and Communit...,11
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 4--- (Goal 4: Quality Education) This ...,4
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 17---\n\n(This answer is based on the ...,17
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---,16


In [21]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,9,9,0,9,9,0,0,0,9,9,9,9,9
1,74,74,74,74,74,74,74,74,74,74,74,74,74
10,33,33,33,33,33,33,33,33,33,33,33,33,33
11,86,86,75,86,86,75,75,75,86,86,86,86,86
12,55,55,54,55,55,54,54,54,55,55,55,55,55
13,61,61,60,61,61,60,60,60,61,61,61,61,61
14,55,55,55,55,55,55,55,55,55,55,55,55,55
15,59,59,57,59,59,57,57,57,59,59,59,59,59
16,49,49,45,49,49,45,45,45,49,49,49,49,49


In [22]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [23]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.15      0.26        60
           1       0.68      0.83      0.75        60
           2       0.93      0.83      0.88        60
           3       0.73      0.95      0.83        60
           4       0.86      0.90      0.88        60
           5       0.85      0.95      0.90        60
           6       0.98      0.83      0.90        60
           7       0.85      0.92      0.88        60
           8       0.53      0.73      0.62        60
           9       0.76      0.75      0.76        60
          10       0.85      0.47      0.60        60
          11       0.67      0.97      0.79        60
          12       0.85      0.78      0.82        60
          13       0.79      0.80      0.79        60
          14       0.96      0.88      0.92        60
          15       0.90      0.88      0.89        60
          16       0.90      0.73      0.81        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# AGGIUNTO IL 2024.05.07:

In [25]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [26]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (This text pertains to infrastru...,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11--- (Sustainable Cities and Communit...,11
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16


In [27]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.39      0.18      0.25        60
           1       0.68      0.83      0.75        60
           2       0.93      0.83      0.88        60
           3       0.73      0.95      0.83        60
           4       0.86      0.90      0.88        60
           5       0.85      0.95      0.90        60
           6       0.98      0.83      0.90        60
           7       0.85      0.92      0.88        60
           8       0.53      0.73      0.62        60
           9       0.76      0.75      0.76        60
          10       0.85      0.47      0.60        60
          11       0.67      0.97      0.79        60
          12       0.85      0.78      0.82        60
          13       0.79      0.80      0.79        60
          14       0.96      0.88      0.92        60
          15       0.90      0.88      0.89        60
          16       0.90      0.73      0.81        60

    accuracy    

In [28]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [29]:
report_df

,precision,recall,f1-score,support
0,39.285714,18.333333,25.000000,60.000000
1,67.567568,83.333333,74.626866,60.000000
2,92.592593,83.333333,87.719298,60.000000
3,73.076923,95.000000,82.608696,60.000000
4,85.714286,90.000000,87.804878,60.000000
5,85.074627,95.000000,89.763780,60.000000
6,98.039216,83.333333,90.090090,60.000000
7,84.615385,91.666667,88.000000,60.000000
8,53.012048,73.333333,61.538462,60.000000
9,76.271186,75.000000,75.630252,60.000000


In [30]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [31]:
print(precision, recall, accuracy, f1_score)

79.27488374127843 78.82352941176471 78.82352941176471 77.97122888781291


In [32]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

79.3 78.8 78.8 78.0
